In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark_session = SparkSession.Builder().getOrCreate()
sc = spark_session.sparkContext
dataset = spark_session.read.csv('/data/movielens/ratings_100k.csv', header=True)
dataset = dataset.select(
    dataset.userId.astype('int'),
    dataset.movieId.astype('int'),
    dataset.rating.astype('double')
)
from pyspark.mllib.recommendation import Rating
splits = dataset.randomSplit([1.0, 1.0, 1.0])

fold_count = 3
rank = 20
iterations = 5
lambda_ = 0.01

#rdds_list= [df.rdd.map(lambda x: Rating(int(x.userId), int(x.movieId), float(x.rating))) for df in splits]
#rdds_list[0].take(3)

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

fold_count = 3
scores = []

for test_fold_index in range(fold_count):
    test = splits[0]
    train = splits[1].union(splits[2])
    #test = rdds_list[0]
    #train = sc.union(rdds_list[1:])
    als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", rank=rank, maxIter=iterations, regParam=lambda_)
    model = als.fit(train)

    # Evaluate the model by computing the RMSE on the test data
    predictions = model.transform(test)
    predictions = predictions.na.drop()
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    scores.append(rmse)

In [3]:
score = sum(scores) / len(scores)
score

1.31024913445959